# E-Commerce Shipping - Baseline Model Training

이 노트북은 전처리된 데이터를 바탕으로 베이스라인 모델을 학습하고 평가하는 과정을 담고 있습니다.

**주요 내용:**
1. 데이터 로드 및 전처리 (Preprocessing Pipeline)
2. 데이터 분할 (Train/Test Split)
3. 베이스라인 모델 학습 (Logistic Regression, Decision Tree, Random Forest)
4. 모델 평가 (Accuracy, F1-score, Confusion Matrix)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# 한글 폰트 설정
if os.name == 'nt':
    plt.rc('font', family='Malgun Gothic')
elif os.name == 'posix':
    plt.rc('font', family='AppleGothic')
else:
    print("Unknown OS")

plt.rc('axes', unicode_minus=False)
import warnings
warnings.filterwarnings('ignore')

## 1. 데이터 로드 및 전처리 (Data Loading & Preprocessing)
앞서 `preprocessing.ipynb`에서 정의한 전처리 파이프라인을 적용합니다.

In [ ]:
# 데이터 로드
df = pd.read_csv('data/Train.csv')

# ID 삭제
if 'ID' in df.columns:
    df = df.drop('ID', axis=1)

# 타겟 변수 이름 변경
if 'Reached.on.Time_Y.N' in df.columns:
    df = df.rename(columns={'Reached.on.Time_Y.N': 'target'})

# --- Feature Engineering ---
# 1. 할인율
df['Discount_Rate'] = (df['Discount_offered'] / df['Cost_of_the_Product']) * 100

# 2. 고할인 여부
df['Is_High_Discount'] = (df['Discount_offered'] > 10).astype(int)

# 3. 무게당 비용
df['Cost_per_Weight'] = df['Cost_of_the_Product'] / df['Weight_in_gms']

# 4. 무게 구간화
bins = [0, 2000, 4000, 6000, float('inf')]
labels = ['Low', 'Medium', 'High', 'Very High']
df['Weight_Range'] = pd.cut(df['Weight_in_gms'], bins=bins, labels=labels)

# --- Preprocessing ---
# Encoding
df['Gender'] = df['Gender'].map({'F': 0, 'M': 1})
df['Product_importance'] = df['Product_importance'].map({'low': 0, 'medium': 1, 'high': 2})

categorical_cols = ['Warehouse_block', 'Mode_of_Shipment', 'Weight_Range']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=False)

# Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
numeric_cols = ['Cost_of_the_Product', 'Weight_in_gms', 'Discount_offered', 
                'Prior_purchases', 'Customer_care_calls', 'Customer_rating',
                'Discount_Rate', 'Cost_per_Weight']
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

print("전처리 완료. 데이터 크기:", df.shape)
display(df.head())

## 2. 데이터 분할 (Train/Test Split)
학습 데이터와 검증 데이터로 분리합니다 (80:20).

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('target', axis=1)
y = df['target']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Train set: {X_train.shape}, Valid set: {X_valid.shape}")

## 3. 베이스라인 모델 학습 (Baseline Models)
가장 기본적인 모델인 로지스틱 회귀, 의사결정나무, 랜덤포레스트를 학습시킵니다.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

# 모델 정의
models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42)
}

# 학습 및 평가 함수
def train_evaluate(models, X_train, y_train, X_valid, y_valid):
    results = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_valid)
        
        acc = accuracy_score(y_valid, y_pred)
        f1 = f1_score(y_valid, y_pred)
        
        results[name] = {'Accuracy': acc, 'F1 Score': f1, 'Model': model}
        
        print(f"--- {name} ---")
        print(f"Accuracy: {acc:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(classification_report(y_valid, y_pred))
        print("-" * 30)
        
    return results

results = train_evaluate(models, X_train, y_train, X_valid, y_valid)

## 4. 모델 평가 시각화 (Evaluation Visualization)
각 모델의 혼동 행렬(Confusion Matrix)과 Feature Importance를 시각화합니다.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, (name, result) in enumerate(results.items()):
    model = result['Model']
    y_pred = model.predict(X_valid)
    cm = confusion_matrix(y_valid, y_pred)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i])
    axes[i].set_title(f'{name} Confusion Matrix')
    axes[i].set_xlabel('Predicted')
    axes[i].set_ylabel('Actual')

plt.tight_layout()
plt.show()

In [ ]:
# Random Forest Feature Importance
rf_model = results['Random Forest']['Model']
importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(12, 6))
plt.title("Random Forest Feature Importance")
plt.bar(range(X.shape[1]), importances[indices], align="center")
plt.xticks(range(X.shape[1]), X.columns[indices], rotation=90)
plt.tight_layout()
plt.show()

## 5. 결론 (Conclusion)
다음 단계로는 하이퍼파라미터 튜닝이나 앙상블 모델링을 통해 성능을 개선할 수 있습니다.